<a href="https://colab.research.google.com/github/FredArgoX/ChaoticTest_Keras/blob/main/Ex_01_Image_classification_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source:
[Image classification from scratch
](https://keras.io/examples/vision/image_classification_from_scratch/)

# Image classification from scratch

Description: Training an image classifier from scratch on the Kaggle Cats vs Dogs dataset.

# Introduction

This example shows how to do image classification fro scratch, starting from JPEG image files on disk, without leveraging pre-trained weights or a pre-made Keras Application model. We demonstrate the workflow on the Kaggle Cats vs Dogs binary classification dataset.

We use the image_dataset_from_directory utility to generate the datasets, and we use Keras image preprocessing layers for image standarization and data augmentation.

# Setup

In [1]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

# Load the data: the Cats vs Dogs dataset

### Raw data download
First, let's download the 786M ZIP archive of the raw data

In [2]:
! curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  93.0M      0  0:00:08  0:00:08 --:--:--  109M


In [3]:
!unzip -q kagglecatsanddogs_5340.zip
!ls

 CDLA-Permissive-2.0.pdf      PetImages        sample_data
 kagglecatsanddogs_5340.zip  'readme[1].txt'


Now we have a PetImages folder which contain two subfolders, Cat and Dog. Each subfolder contains image files for each category

In [4]:
!ls PetImages

Cat  Dog


### Filter out corrupted images
When working with lots of real-world image data, corrupted images are a common occurrence. Let's filter out badly-encoded images that do not feature the string "JFIF" in their header.

In [5]:
num_skipped = 0
for folder_name in ("Cat", "Dog"):
  folder_path = os.path.join("PetImages", folder_name)
  for fname in os.listdir(folder_path):
    fpath = os.path.join(folder_path, fname)
    try:
      fobj = open(fpath, "rb")
      is_jfif = b"JFIF" in fobj.peek(10)
    finally:
      fobj.close()

    if not is_jfif:
      num_skipped += 1
      # Delete corrupted image
      os.remove(fpath)

print(f"Deleted {num_skipped} images.")

Deleted 1590 images.


# Generate a dataset

In [ ]:
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

# Visualize the data

# Using image data augmentation

# Standarizing the data

# Configure the dataset for performance

# Build a model

# Train the model

# Run inference on new data